In [1]:
 %load_ext autoreload
 %autoreload 2

In [26]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
from ray_rntk import *
import json

In [3]:
Ti = 2
TiPrime = 2

In [4]:
dataset = "trains"

In [5]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [6]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [7]:
rntk = RNTKTEST(dic)

In [8]:
DATA = T.Placeholder((N, length), 'float32')
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [9]:
# RNTK,GP = RNTK_avg.RNTK.RNTK_first(rntk,DATA[:,0])

In [10]:
l=1

## LAYER ZERO ATTEMPT


In [11]:
import ray
try:
    ray.init()
except:
    ray.shutdown()
    ray.init()

2021-06-21 16:02:36,091	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


In [12]:
switch_flag = 1
dim_1_i = dim_1 = 2
dim_2 = 2
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

actor = RNTKActor.remote(dim_1, dim_2, dic)

for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i
`
    print(f"called {tiprime} {ti}")
    diag_func.remote(actor, rntk, tiprime, ti, dim_1, dim_2, n)

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

called 2 0
called 1 0
called 0 0
called 0 1
called 0 2


In [30]:
phi_r = ray.get(actor.get_phi.remote())
lambda_r = ray.get(actor.get_lambda.remote())
# print(phi_r)
# print(lambda_r)

In [21]:
test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2

In [32]:
type()

In [29]:
with open("test.json", "w") as file:
    json.dump(lambda_r[0,0,0].__str__(), file)

In [17]:
ray.get(actor.get_and_clear_messages.remote())

['inner iteration - 2 - 0',
 'inner iteration - 1 - 0',
 'inner iteration - 0 - 2',
 'inner iteration - 0 - 1',
 'inner iteration - 0 - 0',
 'inner iteration test value - Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)',
 'inner iteration test value - Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)',
 'inner iteration test value - Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)',
 'inner iteration - 1 - 2',
 'inner iteration test value - Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)',
 'inner iteration - 2 - 1',
 'inner iteration test value - Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)',
 'inner iteration - 1 - 1']

In [15]:
asdfadsfa

(pid=58922) inner iteration test value -  Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)
(pid=58922) inner iteration -  2 1
(pid=58921) inner iteration test value -  Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)
(pid=58917) inner iteration test value -  Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)
(pid=58923) inner iteration test value -  Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)
(pid=58923) inner iteration -  1 2
(pid=58913) inner iteration test value -  Op(name=add_1, fn=add, shape=(10, 10), dtype=float32, scope=/default)
(pid=58913) inner iteration -  1 1


NameError: name 'asdfadsfa' is not defined

## MISC LOOKING AT OBJECT FORMATS

In [16]:
S_new,D_new = rntk.VT(gp_new[l-1])

NameError: name 'gp_new' is not defined

In [115]:
S_new

Op(name=multiply_137, fn=multiply, shape=(10, 10), dtype=float32, scope=/default)

In [116]:
D_new

Op(name=true_divide_19, fn=true_divide, shape=(10, 10), dtype=float32, scope=/default)

In [117]:
extra = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + rntk.su**2 * S_new + rntk.sb**2

In [118]:
new = T.expand_dims(extra,axis = 0)

In [119]:
new

Op(name=expand_dims_27, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [120]:
gp_new = T.concatenate([gp_new,new]) #line 4, alg 1

In [121]:
gp_new

Op(name=concatenate_13, fn=concatenate, shape=(3, 10, 10), dtype=float32, scope=/default)

## MIDDLE RNTK OBJECT TYPES

In [122]:
T.transpose(DATA[:, 1:])

Op(name=transpose_10, fn=transpose, shape=(28, 10), dtype=float32, scope=/default)

In [123]:
T.stack([RNTK,GP])

Op(name=stack_10, fn=stack, shape=(2, 2, 10, 10), dtype=float32, scope=/default)

In [124]:
r1, r2 = T.scan(lambda a,b:middletest(a,b),sequences=[ T.transpose(DATA[:, 1:]) ], init=T.stack([RNTK,GP]))

a[0] Op(name=_rewriting_take_56, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
a[1] Op(name=_rewriting_take_57, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
B Placeholder(name=_19, shape=(10,), dtype=float32, scope=/default)


In [125]:
def middletest(a,b):
    # print("A", a)
    print("a[0]", a[0])
    print("a[1]", a[1])
    print("B", b)
    return a,b